In [ ]:
import requests

resp = requests.post(
    'https://openapi.vito.ai/v1/authenticate',
    data={'client_id': '',
          'client_secret': ''}
)
resp.raise_for_status()
print(resp.json())

In [ ]:
!uv pip install python-dotenv requests

In [ ]:
import os
import glob
import json
import requests
import csv
from dotenv import load_dotenv
import time

# === 0. 환경 변수 로드 및 JWT 확인 ===
load_dotenv()
jwt_token = os.getenv('VITO_TOKEN')
if not jwt_token:
    raise ValueError("환경변수 'VITO_TOKEN'이 설정되지 않았습니다. .env 파일을 확인하세요.")

# === 1. 경로 및 파일 목록 설정 ===
folder_path = r"C:\Users\user\Desktop\woogawooga\woogawooga_project\dataset"
mp3_files = glob.glob(os.path.join(folder_path, "*.mp3"))  # 폴더 내 mp3 파일 모두 찾기

# === 2. CSV 파일 준비 ===
csv_path = os.path.join(folder_path, "vito_transcription_all.csv")
csv_header = ["file_name", "start_at", "duration", "spk", "msg", "lang"]

# 파일이 없으면 헤더 작성
if not os.path.exists(csv_path):
    with open(csv_path, mode="w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(csv_header)

# === 3. 파일별 반복 처리 ===
for mp3_file_path in mp3_files:
    file_name = os.path.basename(mp3_file_path)
    print(f"\n[▶] 파일 처리 시작: {file_name}")

    config = {
        "use_diarization": True,
        "diarization": {"enable": True},
        "use_disfluency_filter": False,
        "use_profanity_filter": False,
        "use_multi_channel": False
    }

    # === 전사 요청 ===
    with open(mp3_file_path, "rb") as f:
        response = requests.post(
            "https://openapi.vito.ai/v1/transcribe",
            headers={"Authorization": f"Bearer {jwt_token}"},
            data={"config": json.dumps(config)},
            files={"file": f}
        )

    response.raise_for_status()
    job_id = response.json()["id"]
    print(f"[✔] 전사 요청 완료 - Job ID: {job_id}")

    # === 상태 확인 ===
    status_url = f"https://openapi.vito.ai/v1/transcribe/{job_id}"
    utterances = None

    while True:
        status_resp = requests.get(status_url, headers={"Authorization": f"Bearer {jwt_token}"})
        status_resp.raise_for_status()
        status_data = status_resp.json()

        if status_data["status"] == "completed":
            if "results" in status_data and "utterances" in status_data["results"]:
                utterances = status_data["results"]["utterances"]
                break
            else:
                time.sleep(5)
                continue
        elif status_data["status"] == "failed":
            print(f"[X] 전사 실패: {file_name}")
            utterances = []
            break
        time.sleep(3)

    # === 추가 대기 (안정화) ===
    time.sleep(5)

    # === 필요시 /result 엔드포인트 ===
    if utterances is None:
        result_url = f"https://openapi.vito.ai/v1/transcribe/{job_id}/result"
        for attempt in range(10):
            result_resp = requests.get(result_url, headers={"Authorization": f"Bearer {jwt_token}"})
            if result_resp.status_code == 200:
                result_json = result_resp.json()
                utterances = result_json["results"]["utterances"]
                break
            time.sleep(5)
        else:
            print(f"[X] /result에서도 실패: {file_name}")
            continue

    # === utterances를 기준으로 CSV 저장 ===
    with open(csv_path, mode="a", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        for utt in utterances:
            writer.writerow([
                file_name,
                utt.get("start_at", ""),
                utt.get("duration", ""),
                utt.get("spk", ""),
                utt.get("msg", ""),
                utt.get("lang", "")
            ])
        print(f" 완료: {file_name}")

print("\n[✔] 모든 파일 처리 및 통합 저장 완료!")


In [ ]:
import pandas as pd

df = pd.read_csv(r"C:\Users\user\Desktop\woogawooga\woogawooga_project\dataset\vito_transcription_all.csv", encoding='utf-8')
df.info()

In [ ]:
import os
import pandas as pd

folder_path = r"C:\Users\user\Desktop\woogawooga\woogawooga_project\dataset"
csv_path = os.path.join(folder_path, "vito_transcription_all.csv")

# CSV 파일이 존재하는지 확인
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    # 이미 처리된 파일명 목록 추출
    processed_files = df['file_name'].unique().tolist()
    print("이미 전사 완료된 파일 개수:", len(processed_files))
    print("전사 완료된 파일 목록(상위 10개):")
    for f in processed_files[:10]:
        print(f)
else:
    print("아직 전사된 파일이 없습니다.")

In [ ]:
import glob

mp3_files = glob.glob(os.path.join(folder_path, "*.mp3"))
mp3_file_names = [os.path.basename(f) for f in mp3_files]

# 미처리 파일 목록
unprocessed_files = list(set(mp3_file_names) - set(processed_files))
print("아직 전사되지 않은 파일 개수:", len(unprocessed_files))
print("미처리 파일 목록(상위 10개):")
for f in unprocessed_files[:10]:
    print(f)


In [ ]:
import os
import glob
import pandas as pd

folder_path = r"C:\Users\user\Desktop\woogawooga\woogawooga_project\dataset"
csv_path = os.path.join(folder_path, "vito_transcription_all.csv")

# 1. 폴더 내 mp3 파일 전체 목록
mp3_files = glob.glob(os.path.join(folder_path, "*.mp3"))
mp3_file_names = sorted([os.path.basename(f) for f in mp3_files])
print("=== mp3 파일 전체 목록 ===")
for name in mp3_file_names:
    print(name)
print(f"\n총 {len(mp3_file_names)}개\n")

# 2. 이미 전사된 파일 목록
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    processed_files = sorted(df['file_name'].unique().tolist())
    print("=== 이미 전사된 파일 목록 ===")
    for name in processed_files:
        print(name)
    print(f"\n총 {len(processed_files)}개\n")
else:
    processed_files = []
    print("전사된 파일이 없습니다.\n")

# 3. 아직 전사되지 않은 파일 목록
unprocessed_files = sorted(list(set(mp3_file_names) - set(processed_files)))
print("=== 아직 전사되지 않은 파일 목록 ===")
for name in unprocessed_files:
    print(name)
print(f"\n총 {len(unprocessed_files)}개")

In [ ]:
import os
import glob
import pandas as pd

folder_path = r"C:\Users\user\Desktop\woogawooga\woogawooga_project\dataset"
csv_path = os.path.join(folder_path, "vito_transcription_all.csv")

# 1. 폴더 내 mp3 파일 전체 목록
mp3_files = glob.glob(os.path.join(folder_path, "*.mp3"))
mp3_file_names = set([os.path.basename(f) for f in mp3_files])

# 2. 이미 전사된 파일 목록
if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    processed_files = set(df['file_name'].unique().tolist())
else:
    processed_files = set()

# 3. 변환(전사)된 mp3 파일만 삭제
files_to_delete = mp3_file_names & processed_files

for name in files_to_delete:
    file_path = os.path.join(folder_path, name)
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"삭제 완료: {name}")

print(f"\n총 {len(files_to_delete)}개 mp3 파일이 삭제되었습니다.")

In [ ]:
df["msg"].str.len().describe()

In [ ]:
df

In [ ]:
df[["spk", "msg"]][55:65]